# GLM


- GLM example as simple multivariate posterior estimation problem with available ground-truth (MCMC)
- setup and code from NIPS 2017 paper

In [ ]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.utils.io as io
import delfi.summarystats as ds
import lfimodels.glm.utils as utils
import matplotlib.pyplot as plt
import numpy as np

from lfimodels.glm.GLM import GLM
from lfimodels.glm.GLMStats import GLMStats
from delfi.utils.viz import plot_pdf

import timeit

%matplotlib inline

seed = 42

# load setup from NIPS 2017 paper

len_filter = 9 # number of GLM filter parameters (= dim. of parameters)
duration = 100 # simulation length (longer = tighter posteriors)

true_params, labels_params = utils.obs_params(len_filter)
obs = utils.obs_data(true_params, seed=seed, duration=duration)
obs_stats = utils.obs_stats(true_params, seed=seed)

# basic approach to controlling generator seeds
def init_g(seed):
    m = GLM(seed=seed, duration=duration, len_filter=len_filter)
    #p = dd.Uniform(lower=[-3,-3,-3,-3,-3,-3,-3,-3,-3,-3], upper=[3,3,3,3,3,3,3,3,3,3])
    p = utils.smoothing_prior(n_params=m.n_params, seed=seed)
    s = GLMStats(n_summary=m.n_params)
    return dg.Default(model=m, prior=p, summary=s)

# MCMC comparison (this might take a while the first time !)
rerun = False  # if False, will try loading file from disk
try:
    assert rerun == False, 'rerun requested'
    sam = np.load('sam.npz')['arr_0']
except:
    sam = utils.pg_mcmc(true_params, obs)
    np.savez('sam.npz', sam)
    

# SNPE parameters

# training schedule
n_train=1000
n_rounds=5

# fitting setup
minibatch=100
epochs=1000

# network setup
n_hiddens=[10,10]
reg_lambda=0.01

# convenience
pilot_samples=1000
svi=False
verbose=True
prior_norm=False

# SNPE-C parameters
n_null = 1

# MAF parameters
mode='random' # ordering of variables for MADEs
n_mades = 2 # number of MADES
act_fun = 'tanh'
batch_norm = False # batch-normalization currently not supported
train_on_all = False # now supported feature

# MDN parameters
n_components = 1

## baseline: SNPE-A

- SNPE-A hard to beat on this problem with Gaussian prior and Gaussian $q^*$.

In [ ]:
g = init_g(seed=seed)

res = infer.CDELFI(g, 
                 obs=obs_stats, 
                 n_hiddens=n_hiddens, 
                 seed=seed, 
                 reg_lambda=reg_lambda,
                 pilot_samples=pilot_samples,
                 svi=svi,
                 n_components=n_components,
                 verbose=verbose,
                 prior_norm=prior_norm)

t = timeit.time.time()

logs_A, tds_A, posteriors_A = res.run(n_train=n_train, 
                    n_rounds=5, 
                    minibatch=minibatch, 
                    epochs=epochs)

print(timeit.time.time() -  t)


# quick look at problem setup, posterior vs. prior

In [ ]:
fig,_ = plot_pdf(posteriors_A[-1], 
         #pdf2=g.prior,
         samples=sam, 
         gt=true_params, 
         resolution=100,
         ticks=True,
         figsize=(16,16));
fig.suptitle('final posterior estimate vs MCMC samples and prior', fontsize=14)
fig.show()


## SNPE-C

- version with rounds: first round is SNPE-A, then 

    - after every round, set $\tilde{p}(\theta) = q^*(\theta|x_0)$
    
    - sample synthetic data set $\mathcal{D} = \{(\theta_n,x_n)\}_{n=1}^N$  for this round, with $(\theta_n, x_n) \sim p(x|\theta)\tilde{p}(\theta)$
    
    - for every gradient step, sample alternatives $\theta'_{nj}, j = 1 \ldots, n_{null}$ from $\theta'_{nj}$ depending on chosen rule: 
        - moo='resample' : $\ \theta'_{nj}\sim Unif[\{\theta_m\}_{n\neq{}m}]$, i.e. $\theta'_{nj}$ are resampled (without replacement) from the $\theta_n\sim \tilde{p}(\theta)$ in the same minibatch. 
        - moo='prior' : $\ \theta'_{nj}\sim p(\theta)$ 
        - moo='p_tilda' : $\ \theta'_{nj}\sim \tilde{p}(\theta)$ with _fixed_ $\tilde{p}(\theta) = q^*(\theta|x_0)$  (default)
        - moo='q_phi_xo': $\ \theta'_{nj}\sim q^*(\theta | x_o)$ with _current_ $q^*$
        - moo='q_phi_x' : $\ \theta'_{nj}\sim q^*(\theta | x_n)$ with _current_ $q^*$
        - note that the two 'q_phi' rules violate the view of drawing $\theta_n, \theta'_{nj}$ iid ! atm it is not perfectly clear what they implement (prbly some form of SVI).
    
    - construct uniform discrete proposals over $\{\theta_n\} \cup \{\theta'_{nj}\}_{j=1}^{n_{null}}$ and compute SNPE-C loss and gradients
    
    - do SGD with minibatches over $minibatch$ many $(\theta_n, x_n)$
    
    
- takes longer than SNPE-A because 

    - for every batch, need to sample $minibath * n_{null}$ many $\theta'_{nj}$ from $minibatch$ many different MoGs $q^*(\theta|x_n)$ !
    - for every batch, need to evaluate $minibath * n_{null}$ many terms $q^*(\theta'_{nj} | x_n)$ (versus $minibatch$ many for SNPE-A)


In [ ]:
if train_on_all:
    epochs = [epochs//(r+1) for r in range(n_rounds)]

# control MAF seed
rng = np.random
rng.seed(seed)

# generator
g = init_g(seed=seed)

# inference object
res_C = infer.SNPEC(g,
                 obs=obs_stats,
                 n_hiddens=n_hiddens,
                 seed=seed,
                 reg_lambda=reg_lambda,
                 pilot_samples=pilot_samples,
                 svi=svi,
                 n_mades=n_mades, # providing this argument triggers usage of MAFs (vs. MDNs)
                 act_fun=act_fun,
                 mode=mode,
                 rng=rng,
                 batch_norm=batch_norm,
                 verbose=verbose,
                 prior_norm=prior_norm)

# train
t = timeit.time.time()

logs_C, tds_C, posteriors_C = res_C.run(
                    n_train=n_train,
                    proposal='discrete',
                    moo='p_tilda',
                    n_null = n_null,
                    n_rounds=n_rounds,
                    train_on_all=train_on_all,
                    minibatch=minibatch,
                    epochs=epochs)

print(timeit.time.time() - t)


In [ ]:
for r in range(n_rounds):
    plt.plot(logs_C[r]['loss'])
    plt.show()

# round-by-round comparison of posterior estimates
- on the first round, SNPE-C = SNPE-A ! (want samples from $q^*_{F(\phi,x_0)}( \cdot{} | x_O)$ to not be much worse than samples from prior, which they are for initial $\phi$ !) 

In [ ]:
for r in range(len(logs_C)):
    
    posterior_A = posteriors_A[-1]
    posterior_C = posteriors_C[r]
    posterior_C.ndim = posterior_A.ndim
    
    fig,_=plot_pdf(posterior_A, 
                   samples=posterior_C.gen(1000).T,
                   gt=true_params, 
                   resolution=100,
                   figsize=(16,16));
    
    fig.suptitle('SNPE-C (samples) vs SNPE-A (density levels), round r = '+str(r+1), fontsize=14)


# SNL

In [ ]:
import sys
import snl.inference.nde as nde
from snl.ml.models.mafs import ConditionalMaskedAutoregressiveFlow
from delfi.utils.delfi2snl import SNLprior, SNLmodel


# control MAF seed
rng = np.random
rng.seed(seed)

# explicit call to MAF constructor
theta, x = g.gen(1)
n_inputs, n_outputs  = x.size, theta.size
model = ConditionalMaskedAutoregressiveFlow(
                n_inputs=n_inputs,
                n_outputs=n_outputs,
                n_hiddens=n_hiddens,
                act_fun=act_fun,
                n_mades=n_mades,
                mode=mode,
                rng=rng)


# generator
g = init_g(seed=seed)

# inference object
inf = nde.SequentialNeuralLikelihood(SNLprior(g.prior),               # method to draw parameters  
                                     SNLmodel(g.model, g.summary).gen # method to draw summary stats
                                    )

# train
t = timeit.time.time()

rng = np.random # control  
rng.seed(seed)  # MCMC seed
model = inf.learn_likelihood(obs_stats.flatten(), model, n_samples=n_train, n_rounds=n_rounds, 
                             train_on_all=False, thin=10, save_models=False, 
                             logger=sys.stdout, rng=rng)

print(timeit.time.time() - t)


In [ ]:
import snl.inference.mcmc as mcmc
log_posterior = lambda t: model.eval([t, obs_stats.flatten()]) + inf.prior.eval(t)
sampler = mcmc.SliceSampler(x=inf.all_ps[-1][-1], lp_f=log_posterior, thin=10)
ps = sampler.gen(1000)

In [ ]:
posterior_A = posteriors_A[-1]
posterior_C = posteriors_C[r]
posterior_C.ndim = posterior_A.ndim

fig,_=plot_pdf(posterior_A, 
               samples=ps.T,
               gt=true_params, 
               resolution=100,
               figsize=(16,16));
    
fig.suptitle('SNL (samples) vs SNPE-A (density levels), final round)')
fig.show()